In [2]:
!pip install tensorflow

^C


     -------------------------------------- 266.3/266.3 MB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 7.2 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB 3.7 MB/s eta 0:00:00
     -------------------------------------- 133.7/133.7 kB 8.2 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 6.0/6.0 MB 7.7 MB/s eta 0:00:00
     -------------------------------------- 896.6/896.6 kB 8.1 MB/s eta 0:00:00
     ---------------------------------------- 24.4/24.4 MB 5.4 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 4.4/4.4 MB 7.0 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 7.3 MB/s eta 0:00:00
     -------------------------------------- 439.2/439.2 kB 6.8 MB/s eta 0:00:00
     ---------------------------------------- 94.2

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from google.colab import drive
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import os
# drive.mount('/content/drive')

ImportError: cannot import name 'OrderedDict' from 'typing' (c:\Users\vishn\anaconda3\envs\ENV1\lib\typing.py)

In [5]:
print("TensorFlow version:", tf.__version__)

NameError: name 'tf' is not defined

In [4]:
def preprocess_data(file_path):
  scaler = MinMaxScaler()
  data_frame = pd.read_csv(file_path).iloc[:, :-1]
  scaled_data = scaler.fit_transform(data_frame)
  return scaled_data

def read_path(path):
  data = []
  for file in os.listdir(path):
    d = preprocess_data(os.path.join(path, file))
    if d.shape[0] >= 30:
      data.append(np.array(d[:30]))
  return data

all_data = []
all_labels = []
for i in range(1, 4):
  curr_data = read_path(f'/content/drive/MyDrive/data/Gesture-{i}')
  all_data.extend(curr_data)
  all_labels.extend([f'gesture{i}' for _ in range(len(curr_data))])

X = np.array(all_data)
y = np.array(all_labels)
X = X.reshape((*X.shape, 1))

In [5]:
num_classes = 3
encoder = LabelEncoder()
y_one_hot = encoder.fit_transform(y)

y_one_hot
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)


model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2], 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

model_checkpoint = 'ml_model.ckpt'
_callback = tf.keras.callbacks.ModelCheckpoint(filepath=model_checkpoint,
                                                        save_weights_only=True,
                                                        verbose=1)
model.fit(X_train, y_train, epochs=50, validation_split=0.15, callbacks=[_callback])

predictions = model.predict(X_test)

predicted_classes = np.argmax(predictions, axis=1)
# model.save('my_model.pb')
model.save('pretrained')

Epoch 1/50
1/2 [==============>...............] - ETA: 0s - loss: 1.0771 - accuracy: 0.5625
Epoch 1: saving model to ml_model.ckpt
2/2 [==============================] - 2s 624ms/step - loss: 1.0630 - accuracy: 0.4915 - val_loss: 1.0870 - val_accuracy: 0.2727
Epoch 2/50
1/2 [==============>...............] - ETA: 0s - loss: 1.0725 - accuracy: 0.3750
Epoch 2: saving model to ml_model.ckpt
2/2 [==============================] - 0s 130ms/step - loss: 0.9763 - accuracy: 0.4407 - val_loss: 1.1045 - val_accuracy: 0.2727
Epoch 3/50
1/2 [==============>...............] - ETA: 0s - loss: 0.9810 - accuracy: 0.4062
Epoch 3: saving model to ml_model.ckpt
2/2 [==============================] - 0s 72ms/step - loss: 0.9226 - accuracy: 0.5424 - val_loss: 1.0438 - val_accuracy: 0.7273
Epoch 4/50
1/2 [==============>...............] - ETA: 0s - loss: 0.7790 - accuracy: 0.8750
Epoch 4: saving model to ml_model.ckpt
2/2 [==============================] - 0s 89ms/step - loss: 0.8709 - accuracy: 0.7458 - va

In [6]:
!zip -r pretrained.zip pretrained

  adding: pretrained/ (stored 0%)
  adding: pretrained/variables/ (stored 0%)
  adding: pretrained/variables/variables.data-00000-of-00001 (deflated 12%)
  adding: pretrained/variables/variables.index (deflated 60%)
  adding: pretrained/fingerprint.pb (stored 0%)
  adding: pretrained/saved_model.pb (deflated 87%)
  adding: pretrained/assets/ (stored 0%)
  adding: pretrained/keras_metadata.pb (deflated 88%)


In [ ]:
predicted_classes, y_test

(array([2, 1, 1, 1, 0, 2, 0, 1, 0, 2, 2, 1, 1, 2, 2, 1, 1, 2]),
 array([2, 0, 1, 1, 0, 2, 0, 0, 0, 2, 2, 2, 1, 2, 2, 2, 1, 2]))

In [ ]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 28ms/step - loss: 0.6582 - accuracy: 0.7778


[0.6582198143005371, 0.7777777910232544]

In [ ]:
new_model = tf.keras.models.load_model('my_model.keras')

# Show the model architecture
preds = new_model.predict(X_test)
np.argmax(preds, axis=1)

1/1 [==============================] - 0s 185ms/step


array([2, 1, 2, 2, 0, 2, 2, 1, 1, 0, 2, 1, 0, 0, 1, 0, 0, 0, 1, 0])